In [56]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed

In [57]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [58]:
# Scenario parameters
a_si = [[1,1], [1,0], [0,1]] # don't include [0,0]
b_sj = [[1,1,1], [1,0,1], [1,1,0], [1,0,0], [0,1,1], [0,1,0], [0,0,1]] # don't include [0,0,0]

In [59]:
Scenarios = [[x,y] for x in a_si for y in b_sj]

In [60]:
Scenarios

[[[1, 1], [1, 1, 1]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 1, 0]],
 [[1, 1], [1, 0, 0]],
 [[1, 1], [0, 1, 1]],
 [[1, 1], [0, 1, 0]],
 [[1, 1], [0, 0, 1]],
 [[1, 0], [1, 1, 1]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 1, 0]],
 [[1, 0], [1, 0, 0]],
 [[1, 0], [0, 1, 1]],
 [[1, 0], [0, 1, 0]],
 [[1, 0], [0, 0, 1]],
 [[0, 1], [1, 1, 1]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 1, 0]],
 [[0, 1], [1, 0, 0]],
 [[0, 1], [0, 1, 1]],
 [[0, 1], [0, 1, 0]],
 [[0, 1], [0, 0, 1]]]

In [61]:
num_Scenarios = len(Scenarios)
p_scen = 1/num_Scenarios

In [62]:
# Product Demand
demand = np.random.randint(0,50,(num_Scenarios, Products,Market))

In [63]:
# Cost of opening
f_i = [200, 50]
f_j = [75, 100, 50]

In [64]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [65]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

In [66]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

In [67]:
# Plant Capacities: Bigger capacities for the more expensive ones
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = np.random.randint(800,1000)
Capacities_i[1] = np.random.randint(200,400) 
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = np.random.randint(400, 600)
Capacities_j[1] = np.random.randint(600, 800)
Capacities_j[2] = np.random.randint(200,400)
Capacities_l = np.random.randint(50,100, (Products,Outsourced)) # in terms of products 


In [68]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(10,15, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(15,20, (Products, Outsourced))

In [69]:
Supplier_cost

array([[[12.49897609, 14.11274253],
        [10.73027833, 12.91518231]],

       [[18.        , 18.        ],
        [16.        , 17.        ]]])

In [70]:
# Cost of transporting product m from outsourced facility l to j
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))
#T_O_DC[0][0] = np.random.uniform(10,15,(Distribution))
#T_O_DC[1][1] = np.random.uniform(10,15,(Distribution))

In [71]:
# Cost of shipping product m from outsourced facility l to k
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))
#T_O_MZ[0][0] = np.random.uniform(15,20,(Market))
#T_O_MZ[1][1] = np.random.uniform(15,20,(Market))

In [72]:
# Product volume 
volume = np.random.uniform(2,3,(Products))

In [73]:
# unit cost of lost sales 
lost_sales = np.random.randint(25,50,(Market,Products))

In [74]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [75]:
## Model

In [76]:
grbModel = Model('synthetic')

In [77]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                U_km[s,k,m] = grbModel.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[s,m,l] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for i in range(Manufacturing_plants):
                Q_im[s,m,i] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for s in range(num_Scenarios):  
        for m in range(Products):
            for i in range(Manufacturing_plants):
                for j in range(Distribution):
                    Y_ijm[s,m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for j in range(Distribution):
                for k in range(Market): 
                    Z_jkm[s,m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    T_ljm[s,m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    T_lkm[s,m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                y_lm[s,m,l] = grbModel.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj()
    ModelCons()

In [78]:
def SolveModel():
    
    grbModel.params.OutputFlag = 0
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    v_val_y_lm = grbModel.getAttr('x', y_lm)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict['ObjVal'] = grbModel.objval
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    
    for s in range(num_Scenarios):        
        Summary_dict["Purchasing_" + str(s)] = sum([v_val_V_lm[(s,m,l)] for m in range(Products) for l in range(Outsourced)])    
        Summary_dict["Production_" + str(s)] = sum([v_val_Q_im[(s,m,i)] for m in range(Products) for i in range(Manufacturing_plants)])
        Summary_dict["LostSales_" + str(s)] = sum([v_val_U_km[(s,k,m)] for m in range(Products) for k in range(Market)])
        Summary_dict["OutsourceToDC_" + str(s)] = sum([v_val_T_ljm[(s,m,l,j)] for m in range(Products) for l in range(Outsourced) for j in range(Distribution)])
        Summary_dict["OutsourceToMarket_" + str(s)] = sum([v_val_T_lkm[(s,m,l,k)] for m in range(Products) for l in range(Outsourced) for k in range(Market)]) 
    
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    
    for s in range(num_Scenarios):
        Cost_dict["InHouseShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
        Cost_dict["OutsourceShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
        Cost_dict["Production_" + str(s)] = get_production_cost(s,v_val_Q_im)
        Cost_dict["Purchasing_" + str(s)] = get_purchase_costs(s,v_val_V_lm, v_val_y_lm)
        Cost_dict["LostSales_" + str(s)] = get_lost_cost(s,v_val_U_km)    
    
    return

In [79]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    for s in range(num_Scenarios):
        ship_1 = 0
        ship_2 = 0
        ship_3 = 0
        ship_4 = 0
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                for m in range(Products):
                    ship_1 += Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j]

        for j in range(Distribution):
            for k in range(Market):
                for m in range(Products):
                    ship_2 += Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k]

        for l in range(Outsourced):
            for j in range(Distribution):
                for m in range(Products):
                    ship_3 += T_O_DC[m][l][j]*T_ljm[s,m,l,j]

        for l in range(Outsourced):
            for k in range(Market):
                for m in range(Products):
                    ship_4 += T_O_MZ[m][l][k]*T_lkm[s,m,l,k]
                    
        total_shipment += ship_1 + ship_2 + ship_3 + ship_4

        # Production
        pr_cost = 0
        for i in range(Manufacturing_plants):
            for m in range(Products):
                pr_cost += Manufacturing_costs[i][m]*Q_im[s,m,i]
                
        total_pr_cost += pr_cost

        # Buying from outsource cost
        b_cost = 0
        for l in range(Outsourced):
            for m in range(Products):
                b_cost += Supplier_cost[0][m][l]*V_lm[s,m,l] + Supplier_cost[1][m][l]*y_lm[s,m,l]
                
        total_b_cost += b_cost

        #Lost Sales
        l_cost = 0
        for k in range(Market):
            for m in range(Products):
                l_cost += lost_sales[k][m]*U_km[s,k,m]
                
        total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + p_scen*(total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [80]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(quicksum(Y_ijm[s,m,i,j] for j in range(Distribution)) == Q_im[s,m,i]
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs(quicksum(Z_jkm[s,m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[s,m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[s,m,l,j] for l in range(Outsourced))) for s in range(num_Scenarios)
                        for j in range(Distribution) for m in range(Products))

    grbModel.addConstrs((quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[s,m,l,k] for l in range(Outsourced))) == (demand[s][m][k] - U_km[s,k,m])
                        for s in range(num_Scenarios) for k in range(Market) for m in range(Products))  
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[s,m,l] == quicksum(T_ljm[s,m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for k in range(Market)) for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[s,m,i] for m in range(Products)) <= Scenarios[s][0][i]*Capacities_i[i]*x_i[i] 
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants))
    
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[s,m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s][1][j]*Capacities_j[j]*x_j[j] for s in range(num_Scenarios) for s in range(num_Scenarios)
                        for j in range(Distribution))
    
    grbModel.addConstrs((V_lm[s,m,l] <= (Capacities_l[m][l])) for s in range(num_Scenarios)
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (5 is arbitrary)
    grbModel.addConstrs((V_lm[s,m,l] - (Capacities_l[m][l] - 25)*y_lm[s,m,l])
                         <= 25 for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))   
    
    
    return   


In [81]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(scen, Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[scen, m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[scen,m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[scen,m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[scen,m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(scen, Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q[scen,m,i]
            
    return(np.round(pr_cost))

def get_purchase_costs(scen, V, y):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V[scen,m,l] + Supplier_cost[1][m][l]*y[scen,m,l]
            
    return(np.round(b_cost))

def get_lost_cost(scen,U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U[scen,k,m]
            
    return(np.round(l_cost))

In [82]:
def run_Model():    
        
    SetGurobiModel()
    SolveModel()

In [83]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [84]:
run_Model()

obj val:  1367.255713019407


In [85]:
num_correct = 0
for s in range(num_Scenarios):
    if (Summary_dict['Purchasing_'+str(s)] + Summary_dict['Production_' + str(s) ] + Summary_dict['LostSales_' + str(s)] == 
          np.sum(demand[s])):
        num_correct += 1
num_correct == num_Scenarios

True

In [86]:
Purchasing = [Summary_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production = [Summary_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales = [Summary_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
OutsourceToDC = [Summary_dict['OutsourceToDC_' + str(s)] for s in range(num_Scenarios)]
OutsourceToMarket = [Summary_dict['OutsourceToMarket_' + str(s)] for s in range(num_Scenarios)]

In [87]:
Unit_df = pd.DataFrame(list(zip(Purchasing, Production, LostSales, OutsourceToDC, OutsourceToMarket)), 
             columns = ["Purchasing", "Production", "LostSales", "OutsourceToDC", "OutsourceToMarket"])

In [88]:
Unit_df

,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,165.0,0.0,0.0,0.0
1,0.0,193.0,0.0,0.0,0.0
2,0.0,143.0,0.0,0.0,0.0
3,11.0,212.0,0.0,0.0,11.0
4,0.0,234.0,0.0,0.0,0.0
5,0.0,151.0,0.0,0.0,0.0
6,117.0,86.0,0.0,0.0,117.0
7,0.0,253.0,0.0,0.0,0.0
8,0.0,209.0,0.0,0.0,0.0
9,0.0,156.0,0.0,0.0,0.0


In [89]:
Purchasing_cost = [Cost_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production_cost = [Cost_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales_cost = [Cost_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
InHouseShipping = [Cost_dict['InHouseShipping_' + str(s)] for s in range(num_Scenarios)]
OutsourceShipping = [Cost_dict['OutsourceShipping_' + str(s)] for s in range(num_Scenarios)]

In [90]:
Cost_dict['Opening'] + p_scen*(sum(Purchasing_cost) + sum(Production_cost) + 
                               sum(InHouseShipping) + sum(OutsourceShipping) + sum(LostSales_cost))

1367.047619047619

In [91]:
grbModel.objVal

1367.255713019407

In [92]:
Cost_df = pd.DataFrame(list(zip(Purchasing_cost, Production_cost, LostSales_cost, InHouseShipping, OutsourceShipping)), 
             columns = ["Purchasing", "Production", "LostSales", "InHouseShipping", "OutsourceShipping"])

In [93]:
Cost_df

,Purchasing,Production,LostSales,InHouseShipping,OutsourceShipping
0,0.0,57.0,0.0,193.0,0.0
1,0.0,74.0,0.0,268.0,0.0
2,0.0,54.0,0.0,210.0,0.0
3,118.0,81.0,0.0,357.0,68.0
4,0.0,192.0,0.0,234.0,0.0
5,0.0,135.0,0.0,173.0,0.0
6,1325.0,43.0,0.0,103.0,662.0
7,0.0,252.0,0.0,237.0,0.0
8,0.0,128.0,0.0,411.0,0.0
9,0.0,160.0,0.0,155.0,0.0


In [94]:
Summary_dict["OpenMPs"]

2.0

In [95]:
Summary_dict["OpenDCs"]

3.0

## VSS

In [107]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

In [108]:
nominal_vals = {}

In [109]:
grbModel_forVSS = Model('VSS')

In [133]:
def SetGurobiModel_forVSS(scen, val):
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[m,l] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)      
    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                    T_lkm[m,l,k] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            y_lm[m,l] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj_forVSS()
    ModelCons_forVSS(scen, val)

In [134]:
def SolveModel_forVSS(nominal):
    
    grbModel_forVSS.params.OutputFlag = 0
    grbModel_forVSS.optimize()
    v_val_x_i = grbModel_forVSS.getAttr('x', x_i)
    v_val_x_j = grbModel_forVSS.getAttr('x', x_j)
    print(grbModel_forVSS.objVal)
    if nominal == True:        
        nominal_vals['x_i'] = v_val_x_i
        nominal_vals['x_j'] = v_val_x_j    
    return

In [135]:
# Objective

def SetGrb_Obj_forVSS():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]
                    
    total_shipment += ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]
                
    total_pr_cost += pr_cost

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[m,l] + Supplier_cost[1][m][l]*y_lm[m,l]
                
    total_b_cost += b_cost

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]
                
    total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + (total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel_forVSS.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 

In [136]:
def ModelCons_forVSS(scen, val):
    
    # Network Flow

    grbModel_forVSS.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i]
                        for i in range(Manufacturing_plants) for m in range(Products))

    grbModel_forVSS.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced)))
                        for j in range(Distribution) for m in range(Products))

    grbModel_forVSS.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced))) == (demand[scen][m][k] - U_km[k,m])
                         for k in range(Market) for m in range(Products))  
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel_forVSS.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market))  
                        for m in range(Products) for l in range(Outsourced))    
    
    # Capacity Constraints
    grbModel_forVSS.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= Scenarios[scen][0][i]*Capacities_i[i]*x_i[i] 
                         for i in range(Manufacturing_plants))
    
    grbModel_forVSS.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[scen][1][j]*Capacities_j[j]*x_j[j]  
                        for j in range(Distribution))
    
    grbModel_forVSS.addConstrs((V_lm[m,l] <= (Capacities_l[m][l])) 
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (5 is arbitrary)
    grbModel_forVSS.addConstrs((V_lm[m,l] - (Capacities_l[m][l] - 25)*y_lm[m,l]) <= 25 
                               for m in range(Products) for l in range(Outsourced))  
    
    
    # Fixing x variables
    if val:
        grbModel_forVSS.addConstrs(x_i[i] == nominal_vals['x_i'][i] for i in range(Manufacturing_plants))
        grbModel_forVSS.addConstrs(x_j[j] == nominal_vals['x_j'][j] for j in range(Distribution))
    
    return   

In [137]:
def run_Model_forVSS(nominal,scen,val):    
        
    SetGurobiModel_forVSS(scen,val)
    SolveModel_forVSS(nominal)

In [138]:
run_Model_forVSS(1,0,0)

619.6942043133467


In [139]:
Scenario_obj_vals = []

In [140]:
for s in range(num_Scenarios):
    run_Model_forVSS(0,s,1)
    Scenario_obj_vals += [grbModel_forVSS.objval]

619.6942043133466
3916.022800102035
593.4074741040706
4268.485829897459
741.960396876439
608.5264558438367
3856.127267904446
819.2361404007934
4063.672832733034
620.8881559823747
2752.0292362861865
503.03705979313804
774.7268214635936
4781.1737738388665
3558.6321364501764
2946.4383661310576
2969.6228356319457
3237.409223064449
4388.871315001049
3152.268128708207
3545.726352092097


In [141]:
average_cost = np.average(Scenario_obj_vals)

In [142]:
VSS = average_cost - Summary_dict['ObjVal']

In [143]:
VSS

1143.1231825338602